In [1]:
from azureml import Workspace

# 1. Connect  to Azure ML

In [2]:
ws = Workspace(workspace_id='',
               authorization_token='')

In [3]:
for ds in ws.user_experiments:
    print(ds)

f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.016f510b1f9a4380a4e9039d2cbd5f8d	Experiment on Random Forest
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.0b4370eacb494b7d8f1b1f37635f4675	test
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.0e551b62145641249a7b5d4fb6435a50	Classification-Forest
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.117fa66620364887a42ad3fcc86d2e62	Boosted-Decision-Tree-Retrain
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.1f7976ac805541b29f4ea4d0c9033f86	Neural-Network
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.2442d4101e4a49a6bb3f259a9b624fb7	Prediction-LinearRegression
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.277e5035cf664fceb0bf6c1c240d25d9	Classification-LogisticRegression [Predictive Exp.]
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.3346d53e36be4985ab3057eb31bfb1e0	Classification-NeuralNetwork
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.46983c4e73fb4493a0ead8f7951d1198	Prediction-NeuralNetwork
f4a5c63f0e6a40e2a8b4244226a5c91f.f-id.46e75a02950d48ecbda01588119b9eb2	Boosted-Decision-Tree-Retrain [Predictive Exp.]
f4a5c63f

# 2. Retrain Model by Request-Response

## Get Error 408 because of Time limiation of Azure ML

In [4]:
import urllib
import json 

# https://s3-us-west-2.amazonaws.com/team3assignmentfinal/cleandata-20130101-20170801.csv

data =  {
        "GlobalParameters": {
        "Data source URL": "https://s3-us-west-2.amazonaws.com/team3assignmentfinal/cleandata-20130101-20170801.csv",
}
    }

body = str.encode(json.dumps(data))

url = ''
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

try:
    req = urllib.request.Request(url, body, headers) 
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)     
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())

The request failed with status code: 408
Content-Length: 195
Content-Type: application/json; charset=utf-8
ETag: "ef43ba5004144031bcc08b978f5de148"
Server: Microsoft-HTTPAPI/2.0
x-ms-request-id: 958f7aac-a3ef-4e80-84ed-d7ee4f2a4219
Date: Thu, 17 Aug 2017 22:36:50 GMT
Connection: close




# 3. Retrain Model by Batch Execution Service

In [5]:
# How this works:
#
# 1. Assume the input is present in a local file (if the web service accepts input)
# 2. Upload the file to an Azure blob - you"d need an Azure storage account
# 3. Call BES to process the data in the blob.
# 4. The results get written to another Azure blob.
# 5. Download the output blob to a local file
#
# Note: You may need to download/install the Azure SDK for Python.
# See: http://azure.microsoft.com/en-us/documentation/articles/python-how-to-install/

import urllib.request
import json
import time
from azure.storage.blob import *

def printHttpError(httpError):
    print("The request failed with status code: " + str(httpError.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(httpError.info())

    print(json.loads(httpError.read().decode("utf8", 'ignore')))
    return

def saveBlobToFile(blobUrl, resultsLabel):
    output_file = "retrain-model.csv" # Replace this with the location you would like to use for your output file, and valid file extension (usually .csv for scoring results, or .ilearner for trained models)
    print("Reading the result from " + blobUrl)
    try:
        response = urllib.request.urlopen(blobUrl)
    except urllib.error.HTTPError as error:
        printHttpError(error)
        return

    with open(output_file, "w+") as f:
        f.write(response.read().decode("utf8", 'ignore'))
    print(resultsLabel + " have been written to the file " + output_file)
    return

def processResults(result):
    first = True
    results = result["Results"]
    for outputName in results:
        result_blob_location = results[outputName]
        sas_token = result_blob_location["SasBlobToken"]
        base_url = result_blob_location["BaseLocation"]
        relative_url = result_blob_location["RelativeLocation"]

        print("The results for " + outputName + " are available at the following Azure Storage location:")
        print("BaseLocation: " + base_url)
        print("RelativeLocation: " + relative_url)
        print("SasBlobToken: " + sas_token)

        if (first):
            first = False
            url3 = base_url + relative_url + sas_token
            saveBlobToFile(url3, "The results for " + outputName)
    return


def invokeBatchExecutionService():
    storage_account_name = "" # Replace this with your Azure Storage Account name
    storage_account_key = "" # Replace this with your Azure Storage Key
    storage_container_name = "" # Replace this with your Azure Storage Container name
    connection_string = "DefaultEndpointsProtocol=https;AccountName=" + storage_account_name + ";AccountKey=" + storage_account_key

    api_key = "" # Replace this with the API key for the web service
    url = ""


    payload = {

            "Outputs": {
                    "output2":
                    {
                        "ConnectionString": connection_string,
                        "RelativeLocation": "/" + storage_container_name + "/retrain-model.csv" # Replace this with the location you would like to use for your output file, and valid file extension (usually .csv for scoring results, or .ilearner for trained models)
                    },
            },

        "GlobalParameters": {
            "Data source URL": "https://s3-us-west-2.amazonaws.com/team3assignmentfinal/cleandata-20130101-20170801.csv",
        }
    }

    body = str.encode(json.dumps(payload))
    headers = { "Content-Type":"application/json", "Authorization":("Bearer " + api_key)}
    print("Submitting the job...")

    # submit the job
    req = urllib.request.Request(url + "?api-version=2.0", body, headers)

    try:
        response = urllib.request.urlopen(req)
    except urllib.error.HTTPError as error:
        printHttpError(error)
        return

    result = response.read()
    job_id = result.decode("utf8", 'ignore')[1:-1]
    print("Job ID: " + job_id)

    # start the job
    print("Starting the job...")
    body = str.encode(json.dumps({}))
    req = urllib.request.Request(url + "/" + job_id + "/start?api-version=2.0", body, headers)
    try:
        response = urllib.request.urlopen(req)
    except urllib.error.HTTPError as error:
        printHttpError(error)
        return

    url2 = url + "/" + job_id + "?api-version=2.0"
    
    consumed_time = 0
    while True:
        print("Checking the job status...")
        req = urllib.request.Request(url2, headers = { "Authorization":("Bearer " + api_key) })

        try:
            response = urllib.request.urlopen(req)
        except urllib.error.HTTPError as error:
            printHttpError(error)
            return

        result = json.loads(response.read().decode("utf8", 'ignore'))
        status = result["StatusCode"]
        if (status == 0 or status == "NotStarted"):
            print("Job " + job_id + " not yet started...")
        elif (status == 1 or status == "Running"):
            print("Job " + job_id + " running...")
        elif (status == 2 or status == "Failed"):
            print("Job " + job_id + " failed!")
            print("Error details: " + result["Details"])
            break
        elif (status == 3 or status == "Cancelled"):
            print("Job " + job_id + " cancelled!")
            break
        elif (status == 4 or status == "Finished"):
            print("Job " + job_id + " finished!")
            processResults(result)
            break
        time.sleep(20) # wait one second
        consumed_time = consumed_time + 20;
        print("The Job has been running for " + str(consumed_time) + "s... You need to wait for almost 20min")
    return

invokeBatchExecutionService()

Submitting the job...
Job ID: f61b087ce588470d86ae82a88e72c488
Starting the job...
Checking the job status...
Job f61b087ce588470d86ae82a88e72c488 not yet started...
The Job has been running for 20s... You need to wait for almost 20min
Checking the job status...
Job f61b087ce588470d86ae82a88e72c488 running...
The Job has been running for 40s... You need to wait for almost 20min
Checking the job status...
Job f61b087ce588470d86ae82a88e72c488 running...
The Job has been running for 60s... You need to wait for almost 20min
Checking the job status...
Job f61b087ce588470d86ae82a88e72c488 running...
The Job has been running for 80s... You need to wait for almost 20min
Checking the job status...
Job f61b087ce588470d86ae82a88e72c488 running...
The Job has been running for 100s... You need to wait for almost 20min
Checking the job status...
Job f61b087ce588470d86ae82a88e72c488 running...
The Job has been running for 120s... You need to wait for almost 20min
Checking the job status...
Job f61b08

The results for output2 have been written to the file retrain-model.csv
